In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("nasdaq_screener.csv")
data.drop(columns=["Last Sale","Country","Net Change","% Change","Market Cap","IPO Year","Volume"],inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
row_lists = data.apply(lambda row: row.tolist(), axis=1).tolist()
print(data)

     Symbol                                               Name  \
0         A             Agilent Technologies Inc. Common Stock   
1        AA                    Alcoa Corporation Common Stock    
2       AAC  Ares Acquisition Corporation Class A Ordinary ...   
3      AACG   ATA Creativity Global American Depositary Shares   
4      AACI            Armada Acquisition Corp. I Common Stock   
...     ...                                                ...   
7052   ZVSA             ZyVersa Therapeutics Inc. Common Stock   
7053    ZWS  Zurn Elkay Water Solutions Corporation Common ...   
7054   ZYME                        Zymeworks Inc. Common Stock   
7055   ZYNE          Zynerba Pharmaceuticals Inc. Common Stock   
7056   ZYXI                            Zynex Inc. Common Stock   

                      Sector  \
0                Industrials   
1                Industrials   
2                    Finance   
3     Consumer Discretionary   
4                Industrials   
...            

In [3]:
import sqlite3
from data.function import Categories

def insert_nasdaq(stock,name,sector_id,industry_id):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f'INSERT INTO stock_nasdaq VALUES (null,"{stock.upper()}","{name}",2,{industry_id},{sector_id})')
    conn.commit()
    conn.close()

def insert_sector(sector,industry_id):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f'INSERT INTO sector VALUES (null,"{sector}",null,{industry_id})')
    conn.commit()
    conn.close()

def insert_industry(industry):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f'INSERT INTO industry VALUES (null,"{industry}",null)')
    conn.commit()
    conn.close()

def insert(data):
    """symbol,name,industry,sector"""
    for i in data:
        cat = Categories()
        all_sector = cat.get_all_sector()
        all_industry = cat.get_all_industry()
        if i[2] not in all_industry:
            insert_industry(i[2])
        if i[3] not in all_sector:
            insert_sector(i[3],cat.get_industry_id(i[2]))
        insert_nasdaq(i[0],i[1],cat.get_sector_id(i[3]),cat.get_industry_id(i[2]))

In [4]:
from data.function import Stock,Categories
import yfinance as yf
import datetime
import sqlite3
from sqlite3 import connect
def get_stock_and_crypto_data(name):
        # start = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')# convert string to datetime obj
        # data = yf.download(tickers=name) #get data
        data = yf.download(tickers=name) #get data
        data = data.loc[data["Volume"] != 0 ].drop(["Adj Close"],axis = 1)# ไม่เอาค่า volume = 0
        data["Date"] = data.index.strftime('%Y-%m-%d %X') # convert pandas timestamp to string 
        return data.values.tolist() #return value in type list
def get_stock_id(symbol):
    """return id of stock"""       
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT stock_id FROM stock_nasdaq WHERE symbol = '{symbol}'")
    data = cursor.fetchall()
    conn.close()
    return data[0][0]

def insert_stock(id,data):
        conn = connect('stock.db',timeout=10)#connect to database
        cursor = conn.cursor()
        for i in data:#run sql script(insert)
            cursor.execute(f"INSERT INTO 'stock_nasdaq_price_hour' VALUES (null,{id},'{i[5]}',{i[0]},{i[1]},{i[2]},{i[3]},{i[4]})")
        conn.commit()#commit change to db
        conn.close()#disconnect

In [5]:
for i in  ["MANU"]:
    id = get_stock_id(i)
    data = get_stock_and_crypto_data(i)
    insert_stock(id,data)

[*********************100%***********************]  1 of 1 completed
